---
title: Prefer repartition to coalesce in Spark
authors:
- Ryan Harter (:harter) 
tags:
- Spark
- ATMO
created_at: 2017-03-02
updated_at: 2017-03-02
tldr: When saving data to parquet in Spark/ATMO, avoid using coalesce. 
---

# Introduction

I ran into some Spark weirdness when working on some ETL.
Specifically, when repartitioning a parquet file with `coalesce()`, the parallelism for the entire job (including upstream tasks) was constrained by the number of coalesce partitions.
Instead, I expected the upstream jobs to use all available cores.
We should be limited by the number of file partitions only when its time to actually write the file.

It's probably easier if I demonstrate.
Below I'll create a small example dataframe containing 10 rows.
I'll map a slow function over the example dataframe in a few different ways.
I'd expect these calculations to take a fixed amount of time, since they're happening in parallel.
However, for one example, **execution time will increase linearly with the number of rows**.

## Setup

In [ ]:
import time
from pyspark.sql.types import LongType

path = "~/tmp.parquet"

In [ ]:
sc.defaultParallelism

In [ ]:
def slow_func(ping):
    """Identity function that takes 1s to return"""
    time.sleep(1)
    return(ping)

In [ ]:
def timer(func):
    """Times the execution of a function"""
    start_time = time.time()
    func()
    return time.time() - start_time

In [ ]:
# Example usage:
timer(lambda: slow_func(10))

In [ ]:
def create_frame(rdd):
    return sqlContext.createDataFrame(rdd, schema=LongType())

## Simple RDD

First, let's look at a simple RDD. Everything seems to work as expected here. Execution time levels off to ~3.7 as the dataset increases:

In [ ]:
map(lambda x: timer(lambda: sc.parallelize(range(x)).map(slow_func).take(x)), range(10))

## Spark DataFrame

Let's create a Spark DataFrame and write the contents to parquet without any modification. Again, things seem to be behaving here. Execution time is fairly flat.

In [ ]:
map(lambda x: timer(lambda: create_frame(sc.parallelize(range(x)))\
                                .coalesce(1).write.mode("overwrite").parquet(path)),
    range(10))

## Offending Example

Now, let's map the slow function over the DataFrame before saving. This should increase execution time by one second for every dataset. However, it looks like **execution time is increasing by one second for each row**.

In [ ]:
map(lambda x: timer(lambda: create_frame(sc.parallelize(range(x))\
                                .map(slow_func))\
                                .coalesce(1).write.mode("overwrite").parquet(path)),
    range(10))

## Repartition fixes the issue

Using `repartition` instead of `coalesce` fixes the issue.

In [ ]:
map(lambda x: timer(lambda: create_frame(sc.parallelize(range(x))\
                                .map(slow_func))\
                                .repartition(1).write.mode("overwrite").parquet(path)),
    range(10))

In [ ]:
sc.cancelAllJobs()